<a href="https://colab.research.google.com/github/aaronjoseph/NLP_Boiler_Plate/blob/main/RoBERTa_Mat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 RoBERTa is a version of BERT which has been trained on a larger corpus for a longer time to achieve better performance in NLU (natural language understanding) tasks. BERT, in turn, is a transformer model originally proposed by Google Research. A short technical introduction is in order now before we start the analysis.

In [1]:
! pip install transformers

# Loading Roberta Specific Libraries
import tensorflow as tf
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification

In [12]:
import os
import random
import re 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
import plotly.express as px

In [3]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.83.158.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.83.158.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  grpc://10.83.158.218:8470
REPLICAS:  8


In [4]:
model_name = 'roberta-large'
roberta_tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
roberta_seq = TFRobertaForSequenceClassification.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
roberta_seq.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  354310144 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  1051650   
Total params: 355,361,794
Trainable params: 355,361,794
Non-trainable params: 0
_________________________________________________________________


In [6]:
twitter = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training.1600000.processed.noemoticon.csv',
                      encoding = 'latin1',
                      names=['polarity','id','date','flag','user','text'])

In [7]:
df = twitter[['polarity','text']]

In [15]:
temp = [(x,y) for x,y in zip(list(df['text']), list(df['polarity']))]
random.shuffle(temp)
tweets = [t[0] for t in temp]
y = [t[1] for t in temp]
y = np.array(y).astype('float32')

In [17]:
# Contains unicode codes (e.g. emojis)?
for t in tweets:
  if 'U+' in t:
    print(t)

# Contains HTML character entities?
for t in tweets:
  if '&' in re.sub(r'(&amp|&gt|&lt)','',t):
    print(t)

Streaming output truncated to the last 5000 lines.
yahhhhh! My &quot;play_all.m3u&quot; file works 
@stevieenglish Would love to offer to help, but my hair is regularly cut with a #2 making &quot;styling&quot; a challenge... 
@TomVMorris &quot;Tom, cloak on&quot; @ponet BOO 
&quot;9&quot; trailer looks cool and steampunk(ish) http://bit.ly/LnJj8 
&quot;Up&quot; review: The opening was pure genius. Looked to be a classic, but the plot got too kiddy for me. In a nutshell: Dogs flying airplanes 
@sandramagdalena Like it tooooo, and also &quot;Wonderful&quot; by Gary Go!! Best summer-song ever 
@Blogography Twitter had that before, if a &quot;GaryLaPointe&quot; went through the tweetverse I'd get a text on the phone, but they took away   
@Syl2010 O twitter &quot;roubou&quot; meu perfil! Tive de refazÃª-lo... snif. Saudades tb! Beijo grande, me twitta! 
They just called my cat &quot;the mean cat&quot; and thought I couldn't hear!  She's not mean, just sad &amp; hurt. I'd be angry too!
&quo

In [18]:
for t in tweets:
  if any([x in t for x in [' btw ',' omg ',' lol ',' thx ']]):
    print(t)

Streaming output truncated to the last 5000 lines.
@HAMMER32 @anna8687 awwwww thx u guys!  I miss your smuttness too.
Time for exams lol cant pee or eat because of them 
@lcharliep3l lol i had salad for dinner. it gave me badbad onion breath for choir  no onion for next time.
@hxcfairy lol we all have different tastes  but not where Mars are concerned lol #30secondstomars
@10marion what's up good lookin? Why u gotta go soundin all sexy on SayNow??  lol   Haven't you learned by now that people are gonna hate?
@jillian79 I'm sooo outta touch w wraps cos I've noone 2wrap lol  anyway why don't u like zaras?jst curious cos I alwys thot they look gd
@laabeebaaa lol im in bed. i have guests downstairs.. my knee is killing me  damn knee i tell u ... u wtchin the match?
@andyclemmensen ily xoxo thx soo much for saying happy birthday  go on msn!
@colorlesswords I KNEW THAT WOULD SCARE ME DAMMIT. The sound when you click his belly, way too sudden omg 
@hydeism lol no i work at night lah aya 
@Fre

In [19]:
def process_tweets(tweets):
    r = tweets
    r = [re.sub(r'https?://t.co/\w+','',t) for t in r]
    r = [re.sub('&amp;','&',t) for t in r]
    r = [re.sub('&gt;','gt',t) for t in r]
    r = [re.sub('&lt;','lt',t) for t in r]
    return r

tweets = process_tweets(tweets)

In [21]:
temp = roberta_tokenizer(tweets[:5],padding='max_length',max_length=50)
temp.keys()

dict_keys(['input_ids', 'attention_mask'])

In [24]:
df

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [25]:
# Combine all data in a separate list used only for determining the length of sequences
all_tweets = list(df['text'])
# Comment out this line if you don't apply any pre-processing
# all_tweets = process_tweets(all_tweets)
max_len = max([len(t) for t in roberta_tokenizer(all_tweets)['input_ids']])
print(max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (549 > 512). Running this sequence through the model will result in indexing errors


549


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets,y,test_size=0.30)
X_train = roberta_tokenizer(X_train,padding='max_length',max_length=max_len,return_tensors='tf')
X_test = roberta_tokenizer(X_test,padding='max_length',max_length=max_len,return_tensors='tf')

In [ ]:
batch_size = 8
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train),y_train))
train_dataset = train_dataset.batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test),y_test))
test_dataset = test_dataset.batch(batch_size)

In [ ]:
temp_x, temp_y = next(iter(test_dataset))
temp = roberta_seq(temp_x,temp_y)
temp

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-6)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
roberta_seq.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [ ]:
chkpt = './checkpoint'
callback_chkpt = tf.keras.callbacks.ModelCheckpoint(chkpt,
                                              monitor='val_accuracy',
                                              save_weights_only=True,
                                              save_best_only=True,
                                              mode='max')

In [ ]:
history = roberta_seq.fit(train_dataset,epochs=3,
                          validation_data=test_dataset,
                          callbacks=[callback_chkpt])

In [ ]:
roberta_seq.load_weights(chkpt)

In [ ]:
outputs = roberta_seq.predict(test_dataset)
y_pred = outputs[0].argmax(axis=1)

In [27]:
from sklearn.metrics import confusion_matrix, classification_report

print('Confusion matrix:')
print(confusion_matrix(y_test,y_pred,labels=[0,1]))
print()
print('Classification report:')
print(classification_report(y_test,y_pred,labels=[0,1],target_names=[0,4]))

Confusion matrix:


NameError: ignored